In [1]:
import numpy as np 
import pandas as pd 
import pickle

from FDSSC import models, utils
from datasets.datasets import LocationChecker, FDSSCDataset
from matplotlib import pyplot as plt
import torch
from torch.utils.data import DataLoader

In [2]:
datadir = "data/stacked_images"
csv = "miningSites/labeled_masked_labels.csv"
checker = LocationChecker(datadir, size=(9, 9), rigorous=False)
landsat8Data = FDSSCDataset(datadir, csv, lochecker=checker)
dataloader = DataLoader(landsat8Data, batch_size=16, shuffle=True, num_workers=1)
#id2idx = utils.get_id2idx(pd.read_csv(csv), "MAJOR_COMMODITY_CODE")
id2idx = pickle.load(open("data/id2idxMasked.p", "rb"))
idx2id = {value: key for key, value in id2idx.items()}

In [4]:
#splitting model to 3 files
parent = "siteData/allData/"
train, val, test = utils.splitData(pd.read_csv(csv).sample(frac=1))
train.to_csv(parent+"train.csv", index=False)
val.to_csv(parent+"val.csv", index=False)
test.to_csv(parent+"test.csv", index=False)

In [6]:
# training model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.FerDSSC_model((1, 9, 9, 11), len(list(id2idx.keys())))
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
model.to(device)

FerDSSC_model(
  (input_spec_conv): Conv3d(1, 24, kernel_size=(1, 1, 7), stride=(1, 1, 1), padding=(0, 0, 3))
  (spectral_conv1): Spectral_conv(
    (bn_prelu): Bn_prelu(
      (bn): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
    )
    (conv3d): Conv3d(24, 12, kernel_size=(1, 1, 7), stride=(1, 1, 1), padding=(0, 0, 3))
  )
  (spectral_conv2): Spectral_conv(
    (bn_prelu): Bn_prelu(
      (bn): BatchNorm3d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
    )
    (conv3d): Conv3d(36, 12, kernel_size=(1, 1, 7), stride=(1, 1, 1), padding=(0, 0, 3))
  )
  (spectral_conv3): Spectral_conv(
    (bn_prelu): Bn_prelu(
      (bn): BatchNorm3d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
    )
    (conv3d): Conv3d(48, 12, kernel_size=(1, 1, 7), stride=(1, 1, 1), padding=(0, 0, 3))
  )
  (bn_prelu1): Bn

In [7]:
utils.train(model, dataloader, epochs=1, loss_fn=loss_fn, optimizer=optimizer)


======== Epoch 1 / 1 ========
Training...
  Batch   194  of    195.    Loss: 4.45     Elapsed: 0:05:22.
  Average training loss: 4.44
  Training epoch took: 0:05:22


[4.438143434280004]

In [ ]:
#callback for